# Stacked Bidirectional LSTM for Sentiment Classification

This Stacked Bi-LSTM performs sentiment analysis on the IMDB review dataset.

In [ ]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, Flatten, Dropout, Activation, SpatialDropout1D
from keras.layers.wrappers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from sklearn.metrics import roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### Set Hyperparameters

In [ ]:
output_dir = 'model_output/stackedbilstm'

epochs = 4
batch_size = 64

n_dim = 128
n_unique_words = 20000 
max_review_length = 300
pad_type = trunc_type = 'pre'
drop_embed = 0.2 

n_lstm_1 = 64
n_lstm_2 = 128
drop_lstm = 0.2

#### Load Data

In [ ]:
(X_train, y_train), (X_valid, y_valid) = imdb.load_data(num_words=n_unique_words)

#### Preprocess Data

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
X_valid = pad_sequences(X_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

#### Design Deep Net Architecture

In [ ]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))
model.add(Bidirectional(LSTM(n_lstm_1, dropout=drop_lstm, return_sequences=True)))
model.add(Bidirectional(LSTM(n_lstm_2, dropout=drop_lstm, return_sequences=True)))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid', name='dense_2'))

In [ ]:
model.summary()

#### Configure the Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
modelCheckpoint = ModelCheckpoint(filepath=output_dir+'/weights-multicnn-toxicity_new.hdf5', save_best_only=True, mode='min')
earlyStopping = EarlyStopping(mode='min', patience=1)

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Train the Model

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.3, callbacks=[modelCheckpoint])

#### Evaluate

In [ ]:
#model.load_weights(output_dir+'/weights.03.hdf5')
print('Not loading weights.')

In [ ]:
y_hat = model.predict_proba(X_valid)

In [ ]:
y_hat[0]

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat) * 100

In [ ]:
'{:0.2f}'.format(pct_auc)